In [1]:
!pip3 freeze
!pip3 install gym numpy tensorflow keras openAi Matplotlib requests pandas gym deap

#Library for multi-objective optimization algorithms
!pip3 install pymoo

import sys
print('System version',sys.version)

about-time==4.2.1
absl-py==2.1.0
alive-progress==3.1.5
annotated-types==0.7.0
anyio==4.4.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
astunparse==1.6.3
async-lru==2.0.4
attrs==24.2.0
autograd==1.7.0
babel==2.16.0
beautifulsoup4==4.12.3
bleach==6.1.0
certifi==2024.8.30
cffi==1.17.1
charset-normalizer==3.3.2
cloudpickle==3.0.0
cma==3.2.2
colorama==0.4.6
comm==0.2.2
contourpy==1.3.0
cycler==0.12.1
deap==1.4.1
debugpy==1.8.5
decorator==5.1.1
defusedxml==0.7.1
Deprecated==1.2.14
dill==0.3.8
distro==1.9.0
executing==2.1.0
fastjsonschema==2.20.0
filelock==3.15.4
flatbuffers==24.3.25
fonttools==4.53.1
fqdn==1.5.1
fsspec==2024.9.0
gast==0.6.0
google-pasta==0.2.0
grapheme==0.6.0
grpcio==1.66.1
gym==0.26.2
gym-notices==0.0.8
h11==0.14.0
h5py==3.11.0
httpcore==1.0.5
httpx==0.27.2
idna==3.8
ipykernel==6.29.5
ipython==8.27.0
ipywidgets==8.1.5
isoduration==20.11.0
jedi==0.19.1
Jinja2==3.1.4
jiter==0.5.0
json5==0.9.25
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema

Problem Statement: 
You will focus on optimizing traffic management for the three to five road segments in New 
York City. For example: 
1. 5th Ave between 42nd St and 47th St (Manhattan) 
2. Atlantic Ave between Flatbush Ave and Bedford Ave (Brooklyn) 
3. Queens Blvd between Union Tpke and Yellowstone Blvd (Queens) 
4. Grand Concourse between E 161st St and E 170th St (Bronx) 
5. Victory Blvd between Richmond Ave and Clove Rd (Staten Island) 
The traffic management strategy involves controlling traffic signal timings (green, yellow, and 
red light durations) and setting speed limits on these segments. Your task is to develop an 
MOEA (discussed in lectures) that optimizes these parameters to achieve the best trade-off 
between minimizing TTT and FC. 

In [29]:
# Average Speed of a Vehicle Traveled Between End Points dataset
# Dataset
# https://data.cityofnewyork.us/Transportation/DOT-Traffic-Speeds-NBE/i4gi-tjb9/data_preview

import requests
import pandas as pd

# Api key if neede
NYC_OPEN_DATA_API_KEY = 'br55d9sqptry35vxzglajqzfe'
NYC_OPEN_DATA_API_SecretKEY='4sk51l5by3uvyn516za8g2wen7cn1wokbry0e2ayfo1lugq6fl'

# API endpoint
Average_Speed_of_Vehicle_Traveled_Between_End_Points_url = "https://data.cityofnewyork.us/resource/i4gi-tjb9.json"

# Optional: Add parameters to limit the number of rows or filter data
params = {
    # "$limit": 1000,  # Fetch 1000 records.
}

# Fetch the data using a GET request
Average_Speed_response = requests.get(Average_Speed_of_Vehicle_Traveled_Between_End_Points_url, params=params)

# Check if the request was successful (status code 200)
if Average_Speed_response.status_code == 200:
    Average_Speed_of_Vehicle_Traveled_Between_End_Points_data = pd.DataFrame(Average_Speed_response.json())
    
    # Display the first few rows of the DataFrame
    print(Average_Speed_of_Vehicle_Traveled_Between_End_Points_data.head())
else:
    print(f"Error: {Average_Speed_response.status_code}")


    id  speed travel_time status               data_as_of  link_id  \
0  165   58.4          49      0  2024-09-28T08:38:09.000  4616305   
1  390   37.9         103      0  2024-09-28T08:38:09.000  4616209   
2  378  45.36          58      0  2024-09-28T08:38:09.000  4616197   
3  369      0           0   -101  2024-09-28T08:38:09.000  4616282   
4  221  39.76         257   -101  2024-09-28T08:38:09.000  4616333   

                                         link_points  \
0  40.81477,-73.83629 40.81611,-73.83627 40.81784...   
1  40.6152105,-74.157401 40.61231,-74.15362 40.60...   
2  40.6210105,-74.168861 40.6207604,-74.168 40.61...   
3  40.65632,-74.11972 40.655041,-74.12241 40.6544...   
4  40.736971,-73.97477 40.73651,-73.975011 40.735...   

                                   encoded_poly_line  \
0                       isbxFxctaMkGCyIs@_NWgVb@iF`@   
1               at{vFvzrcMbQsVlP{UjCkFbBsFxAkJ\{HKiS   
2                    ix\\|vFjbucMp@kD\\|Ncb@pP_ZzAsB   
3  _ucwFfokcM~FxO\

In [11]:
# Traffic Volume Count dataset
# Dataset
# https://data.cityofnewyork.us/Transportation/Traffic-Volume-Counts/btm5-ppia/data_preview

import requests
import pandas as pd

# Api key if neede
NYC_OPEN_DATA_API_KEY = 'br55d9sqptry35vxzglajqzfe'
NYC_OPEN_DATA_API_SecretKEY='4sk51l5by3uvyn516za8g2wen7cn1wokbry0e2ayfo1lugq6fl'

# API endpoint
Traffic_Volume_Count_url = "https://data.cityofnewyork.us/resource/i4gi-tjb9.json"

# Optional: Add parameters to limit the number of rows or filter data
params = {
    "$limit": 100000,  # Fetch 1000 records.
}

# Fetch the data using a GET request
Traffic_Volume_Count_response = requests.get(Traffic_Volume_Count_url, params=params)

# Check if the request was successful (status code 200)
if Traffic_Volume_Count_response.status_code == 200:
    Traffic_Volume_Count_data = pd.DataFrame(Traffic_Volume_Count_response.json())
    
    # Display the first few rows of the DataFrame
    print(Traffic_Volume_Count_data.head())
else:
    print(f"Error: {Traffic_Volume_Count_response.status_code}")


KeyboardInterrupt: 

    id  speed travel_time status               data_as_of  link_id  \
0  258  24.23         175      0  2024-09-28T07:38:09.000  4616220   
1  351  45.98         115      0  2024-09-28T07:38:09.000  4616210   
2  222  22.99         294   -101  2024-09-28T07:38:09.000  4616332   
3  338      0           0   -101  2024-09-28T07:38:09.000  4616255   
4  390  52.81          74      0  2024-09-28T07:38:09.000  4616209   

                                         link_points  \
0  40.6162405,-74.02612 40.61923,-74.02361 40.623...   
1  40.63092,-74.14592 40.62975,-74.14593 40.62877...   
2  40.7606904,-73.956791 40.7601606,-73.95751 40....   
3  40.81376,-73.93128 40.8127206,-73.9316 40.8121...   
4  40.6152105,-74.157401 40.61231,-74.15362 40.60...   

                                   encoded_poly_line  \
0         oz{vFffybMuQuNmZsVsFyC}D}AeLmDeO_E{JkCaEg@   
1  gv~vF~rpcMhF@bE[bDWbD@vCb@\\|D~@dCf@`Cv@jCjA~C...   
2  iaxwF\\|tkbMhBnCtChChGpEjGxE~CvBxEnGlGhFrA\\|A...   
3  _mbxFnufbMnE~@x

# For selecting the data in the first area:
1. 5th Ave between 42nd St and 47th St (Manhattan) 

In [14]:
# Print column names
print("Column names in the dataset:")
print(Average_Speed_of_Vehicle_Traveled_Between_End_Points_data.columns) 

Column names in the dataset:
Index(['id', 'speed', 'travel_time', 'status', 'data_as_of', 'link_id',
       'link_points', 'encoded_poly_line', 'encoded_poly_line_lvls', 'owner',
       'transcom_id', 'borough', 'link_name'],
      dtype='object')


In [45]:
# Select all rows where borough = "Manhattan", && link_name between 42th, and 47th St.
filter_1 = Average_Speed_of_Vehicle_Traveled_Between_End_Points_data[
    (Average_Speed_of_Vehicle_Traveled_Between_End_Points_data['borough'] == 'Manhattan') 
    & 
    (Average_Speed_of_Vehicle_Traveled_Between_End_Points_data['link_name'].str.contains('57th St')) 

    #(Average_Speed_of_Vehicle_Traveled_Between_End_Points_data['link_name'].isin (['43th St', '44th St', '45th St', '46th St']))
    ]
# Print the filtered results
print(filter_1)

        id speed travel_time status               data_as_of  link_id  \
5        4     0           0   -101  2024-09-28T08:38:09.000  4616338   
43     450     0           0   -101  2024-09-28T08:38:03.000  4616346   
54     106     0           0   -101  2024-09-28T08:38:03.000  4616323   
98     448     0           0   -101  2024-09-28T08:38:03.000  4620343   
127      4     0           0   -101  2024-09-28T08:34:09.000  4616338   
...    ...   ...         ...    ...                      ...      ...   
99871  450     0           0   -101  2024-09-25T12:29:03.000  4616346   
99878  448     0           0   -101  2024-09-25T12:29:03.000  4620343   
99926    4     0           0   -101  2024-09-25T12:24:10.000  4616338   
99929  106     0           0   -101  2024-09-25T12:24:10.000  4616323   
99994  448     0           0   -101  2024-09-25T12:24:03.000  4620343   

                                             link_points  \
5      40.7607,-74.002141 40.76212,-74.91 40.76335,-7...   
43 

# For selecting the data in the second area:
2. Atlantic Ave between Flatbush Ave and Bedford Ave (Brooklyn) 

In [52]:
# Select all rows where Atlantic Ave between Flatbush Ave and Bedford Ave (Brooklyn).
filter_2 = Average_Speed_of_Vehicle_Traveled_Between_End_Points_data[
    (Average_Speed_of_Vehicle_Traveled_Between_End_Points_data['borough'] == 'Brooklyn') 
    #& 
    #(Average_Speed_of_Vehicle_Traveled_Between_End_Points_data['link_name'].isin (['Flatbush Ave', 'Bedford Ave']))
    ]
# Print the filtered results
print(filter_2)

        id  speed travel_time status               data_as_of  link_id  \
13     258  41.63         103      0  2024-09-28T08:38:09.000  4616220   
18     261  21.74         193      0  2024-09-28T08:38:09.000  4616226   
35     154  42.87          95      0  2024-09-28T08:38:03.000  4616229   
38     157  34.17         450      0  2024-09-28T08:38:03.000  4616271   
55     264      0           0   -101  2024-09-28T08:38:03.000  4763655   
...    ...    ...         ...    ...                      ...      ...   
99937  258  44.11          96      0  2024-09-25T12:24:10.000  4616220   
99942  261  34.17         125      0  2024-09-25T12:24:10.000  4616226   
99974  154   49.7          82      0  2024-09-25T12:24:03.000  4616229   
99975  259   4.97         811      0  2024-09-25T12:24:03.000  4616223   
99979  148  27.34         545      0  2024-09-25T12:24:03.000  4616257   

                                             link_points  \
13     40.6162405,-74.02612 40.61923,-74.02361 40.6

# For selecting the data in the third area:
3. Queens Blvd between Union Tpke and Yellowstone Blvd (Queens) 

In [54]:
# Select all rows where Queens Blvd between Union Tpke and Yellowstone Blvd (Queens).
filter_3 = Average_Speed_of_Vehicle_Traveled_Between_End_Points_data[
    (Average_Speed_of_Vehicle_Traveled_Between_End_Points_data['borough'] == 'Queens') 
    & 
    (Average_Speed_of_Vehicle_Traveled_Between_End_Points_data['link_name'].isin (['Union Tpke', 'Yellowstone Blvd']))
    ]
# Print the filtered results
print(filter_3)

Empty DataFrame
Columns: [id, speed, travel_time, status, data_as_of, link_id, link_points, encoded_poly_line, encoded_poly_line_lvls, owner, transcom_id, borough, link_name]
Index: []


In [8]:
# Merge the dataframes on id and post_id
merged_df = pd.merge(Traffic_Volume_Count_data, Average_Speed_of_Vehicle_Traveled_Between_End_Points_data, left_on='id', right_on='post_id', how='inner')

NameError: name 'Average_Speed_of_Vehicle_Traveled_Between_End_Points_data' is not defined